In [ ]:
!pip install -Uq "google-genai==1.7.0"
!pip install faiss-cpu
!pip install hf_xet
!pip install gradio

In [ ]:
from google import genai
from google.genai import types
from IPython.display import Markdown, display
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import os

In [ ]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [ ]:
client = genai.Client(api_key="GOOGLE_API_KEY")

In [ ]:
# PEP 8 Guidelines for RAG
pep8_guidelines = [
    "Use 4 spaces per indentation level.",
    "Function names should be lowercase, with words separated by underscores.",
    "Use blank lines to separate functions and classes.",
    "Keep lines to a maximum of 79 characters.",
    "Always include a docstring for public modules, functions, classes, and methods.",
    "Class names should use the CapWords convention.",
    "Use 'self' as the first parameter of instance methods."
]

In [ ]:
embed_model = SentenceTransformer("all-MiniLM-L6-v2")
pep8_embeddings = embed_model.encode(pep8_guidelines).astype("float32")

index = faiss.IndexFlatL2(pep8_embeddings.shape[1])
index.add(pep8_embeddings)

In [ ]:
def retrieve_pep8_recommendations(query, top_k=3):
    query_vector = embed_model.encode([query]).astype("float32")
    distances, indices = index.search(query_vector, top_k)
    return [pep8_guidelines[i] for i in indices[0]]

In [ ]:
# Code Generator Agent
def generate_code(prompt):
    pep8_tips = retrieve_pep8_recommendations(prompt)
    pep8_text = "\n".join(pep8_tips)
    full_prompt = f"""Write Python code for the following task:

    Task: {prompt}

    Please follow these PEP 8 Guidelines:
    {pep8_text}

    Output only code."""
    response = client.models.generate_content(
        model="gemini-2.0-flash",
        contents=f"Write Python code for the following task:\n{prompt}")
    return response.text.strip('```python').strip('```').strip()

In [ ]:
# Code Executor Agent (Simulated execution)
import traceback

def execute_code(code):
    try:
        exec_globals = {}
        exec(code, exec_globals)
        return "SUCCESS", None
    except Exception as e:
        return "ERROR", traceback.format_exc()

In [ ]:
# Error Fixer Agent
def fix_code(original_code, error_message):
    prompt = f"""The following Python code has an error:
    {original_code}
    Error Message:
    {error_message}
    Output only the fixed code."""

    response = client.models.generate_content(
                model="gemini-2.0-flash",
                contents=prompt)
    return response.text.strip('```python').strip('```').strip()

In [ ]:
# Code Explaining Agent
def explain_code(code):
    prompt = f"""Please explain the following Python code in simple bullet points. Use clear and beginner-friendly language.
   Code:
    ```python
    {code}
    Explanation: """

    response = client.models.generate_content(
        model="gemini-2.0-flash",
        contents=prompt)
    return response.text.strip('```python').strip('```').strip()

In [ ]:
def validate_code(prompt, max_attempts=3):
    code = generate_code(prompt)
    for attempt in range(max_attempts):
        print(f"\n Attempt {attempt + 1}:\n{code}\n")
        status, error = execute_code(code)
        if status == "SUCCESS":
            print("Code executed successfully without errors!")
            explantion=explain_code(code)
            return code
        else:
            print(f"Error Detected:\n{error}")
            code = fix_code(code, error)
    print("Maximum attempts reached. Final code might still have issues.")
    return code

In [ ]:
final_code = validate_code("Write a Python function to calculate factorial of a number input by the user")
print("\n🧾 Final Code:\n", final_code)